In [1]:
#%pip install --quiet llama-index llama-index-llms-gemini llama-index-embeddings-huggingface pydantic-ai 

In [1]:
MODEL_ID = "gemini-2.0-flash"
EMBED_MODEL_ID = "BAAI/bge-small-en-v1.5"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
       "Please specify the GEMINI_API_KEY access token in keys.env file"
assert os.environ["HF_TOKEN"][:2] == "hf",\
       "Please specify the HF_TOKEN access token in keys.env file"

In [2]:
import sys
sys.path.append('../basic_rag')
import gutenberg_text_loader as gtl
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

The examples here on the book Anabasis of Alexander https://www.gutenberg.org/cache/epub/46976/pg46976.txt
a 2nd century historical account of Alexander the Great

## Plain Semantic Indexing to use as a comparison

In [3]:
#!rm -rf .cache vector_index   # uncomment to start afresh

In [4]:
!ls ./.cache vector_index

./.cache:
pg46976_1b16d8525c.txt

vector_index:
default__vector_store.json  graph_store.json	      index_store.json
docstore.json		    image__vector_store.json


In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core import Document
import os
import pathlib

INDEX_DIR="vector_index"
Settings.embed_model = HuggingFaceEmbedding(
    model_name=EMBED_MODEL_ID
)

# these are the defaults in LlamaIndex
Settings.chunk_size = 1024; Settings.chunk_overlap = 20; TOP_K=2
#Settings.chunk_size = 100; Settings.chunk_overlap = 10; TOP_K=4

if os.path.isdir(INDEX_DIR):
    print("Loading in already created index")
    storage_context = StorageContext.from_defaults(persist_dir=INDEX_DIR)
    index = load_index_from_storage(storage_context)
else:
    # downloads into .cache the first time
    gs = gtl.GutenbergSource()
    doc = gs.load_from_url("https://www.gutenberg.org/cache/epub/46976/pg46976.txt")
    # reads all files in .cache
    documents = SimpleDirectoryReader(input_dir="./.cache", required_exts=[".txt"], exclude_hidden=False).load_data()
    # creates a vector db
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=INDEX_DIR)

2025-03-26 18:48:13,073 - INFO - PyTorch version 2.5.1 available.
2025-03-26 18:48:14,440 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2025-03-26 18:48:16,422 - INFO - 2 prompts are loaded, with the keys: ['query', 'text']


Loading in already created index


2025-03-26 18:48:19,662 - INFO - Loading all indices.


In [9]:
from llama_index.llms.gemini import Gemini
from llama_index.core.query_engine import RetrieverQueryEngine

llm = Gemini(model=f"models/{MODEL_ID}", api_key=os.environ["GEMINI_API_KEY"])

query_engine = RetrieverQueryEngine.from_args(
    retriever=index.as_retriever(similarity_top_k=TOP_K), llm=llm,
)

def print_response(question):
    response = query_engine.query(question)
    response = {
        "answer": str(response),
        "source_nodes": response.source_nodes
    }
    print(response['answer'])
    for node in response['source_nodes']:
        print(node)
    return response
 
print_response("How did Alexander treat the people of the places he conquered?");

/var/tmp/ipykernel_44368/3339145799.py:4: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(model=f"models/{MODEL_ID}", api_key=os.environ["GEMINI_API_KEY"])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Alexander took Hyparna at first assault but allowed the Greeks to leave the citadel under a truce. He brought over the Telmissians by agreement. Cities including Pinara, Xanthus, and Patara surrendered to him. He ordered the Phaselites and Lycians to surrender their cities to those he sent to receive them, and they were all surrendered. He helped the men of Phaselis capture a fort constructed by the Pisidians. When Thebes was razed, Alexander preserved the house and descendants of Pindar the poet. He gave a courteous reply to an Athenian embassy. He remitted his wrath against those the Athenians refused to surrender to him, but ordered Charidemus to go into banishment.

Node ID: d2b4750d-8bb7-4b64-97e1-d9e1fd622239
Text: He gave these  officers instructions to levy as many horse and
foot soldiers as they  could from the country, when they returned to
him and brought back the  men who had been sent away with them. By
this act more than by any  other Alexander acquired popularity among
t

Saved answer:
<pre>
Alexander took Hyparna at first assault but allowed the Greeks to leave the citadel under a truce. He brought over the Telmissians by agreement. Cities including Pinara, Xanthus, and Patara surrendered to him. He ordered the Phaselites and Lycians to surrender their cities to those he sent to receive them, and they were all surrendered. He helped the men of Phaselis capture a fort constructed by the Pisidians. When Thebes was razed, Alexander preserved the house and descendants of Pindar the poet. He gave a courteous reply to an Athenian embassy. He remitted his wrath against those the Athenians refused to surrender to him, but ordered Charidemus to go into banishment.
</pre>

In [7]:
print_response("Where did Alexander die?");

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The provided text does not contain information about where Alexander died.

Node ID: 3ae51689-746d-46c7-8742-986d72fc9007
Text: When  he received this news, he stopped the march towards
Bactra, and taking  with him the Companion cavalry, the horse-lancers,
the archers, the  Agrianians and the regiments of Amyntas and Coenus,
and leaving the  rest of his forces there under the command of
Craterus, he made a  forced march against Satibarzanes and the
Areians; and having tr...
Score:  0.699

Node ID: 6089be70-704d-4546-89df-75c34ea6d0a7
Text: Go back and  report at home that your king Alexander, the
conqueror of the Persians,  Medes, Bactrians, and Sacians[875]; the
man who has subjugated the  Uxians, Arachotians, and Drangians; who
has also acquired the rule  of the Parthians, Chorasmians, and
Hyrcanians, as far as the Caspian  Sea; who has marched over the
Caucasus, through the Cas...
Score:  0.695



Saved answer:
<pre>
The provided text does not contain information about where Alexander died.
</pre>

## Limitations of the Semantic Indexing

It's unable to find small details because the details are lost in the larger chunk. This was not a problem for Basic RAG which is keyword-based because Diogenes is quite a rare word, but it is a problem for Semantic Indexing.

See: https://docs.llamaindex.ai/en/stable/optimizing/basic_strategies/basic_strategies/

A smaller chunk size means the embeddings are more precise, while a larger chunk size means that the embeddings may be more general, but can miss fine-grained details.

Here we show the differences between a chunk size of 100 and 1024

### Fine details
For chunk size = 100:
<pre>
Alexander admired Diogenes' conduct. Diogenes requested that Alexander and his attendants move out of the sunlight.
</pre>
For chunk size = 1024:
<pre>
The provided text does not contain information about Diogenes.
</pre>

In [11]:
print_response("Describe the relationship between Alexander and Diogenes.");

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The provided text does not contain information about Diogenes.

Node ID: 6089be70-704d-4546-89df-75c34ea6d0a7
Text: Go back and  report at home that your king Alexander, the
conqueror of the Persians,  Medes, Bactrians, and Sacians[875]; the
man who has subjugated the  Uxians, Arachotians, and Drangians; who
has also acquired the rule  of the Parthians, Chorasmians, and
Hyrcanians, as far as the Caspian  Sea; who has marched over the
Caucasus, through the Cas...
Score:  0.729

Node ID: 64ea1229-b295-46c5-bcc3-70c8c2f1b017
Text: They were also instructed to support this petition  by word of
mouth. The letter pointed out to him that friendship and  alliance had
subsisted between Philip and Artaxerxes;[284] and that  when Arses,
son of Artaxerxes, ascended the throne, Philip was the  first to
practise injustice towards him, though he had suffered no  injury from
the Persi...
Score:  0.723



### Complex query

This is a complex query that requires multiple supporting points pulled from the chunks. None of the chunks will discuss an overall strategy. Hence, the result here is of a tactic in a single battle.

chunksize = 100
<pre>
Alexander led a quick charge with a wedge formation of Companion cavalry and part of the phalanx straight toward Darius. He also enumerated evidences of their superiority in the struggle and pointed out the great rewards they would win.
</pre>

chunk size = 1024
<pre>
Alexander was advised to advance against Darius and the Persians without delay. He marched his troops towards Darius, but a storm delayed him. Upon hearing that Darius was in his rear, Alexander sent a ship to Issus to confirm the report.
</pre>

In [13]:
print_response("What was Alexander's strategy against Darius III?");

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Upon learning that Darius was in his rear, Alexander encouraged his generals, cavalry commanders, and Grecian leaders to take courage. He pointed out that they, as victors, would be fighting a defeated foe, and that the deity was helping them by leading Darius to move his forces from a spacious plain into a narrow place, negating the advantage of the vast multitude of the opposing army. He also noted the difference in strength, courage, and purpose between his army and that of Darius.

Node ID: d172a7db-ece4-4be3-a9a1-1306501103f2
Text: ALEXANDER’S TACTICS.—HIS SPEECH TO THE OFFICERS.      When
Alexander had received all this information from the Persian  scouts
who had been captured, he remained four days in the place where  he
had received the news; and gave his army rest after the march. He
meanwhile fortified his camp with a ditch and stockade, as he intended
to leave beh...
Score:  0.766

Node ID: fd256fc4-7d16-4f29-a53c-141e245fdcc0
Text: So Darius remained. But as Alexander made

## Hypothetical Document Embedding (HyDE)

Here we show how building a hypothetical answer and then searching for it can help find details that are lost in the chunk, and in pulling data across multiple chunks.

In [14]:
from llama_index.core.llms import ChatMessage
def create_hypothetical_answer(question):
    messages = [
        ChatMessage(role="system", 
                    content="""Answer the following question in 2-3 sentences.
                    If you don't know the answer, make an educated guess."""
                   ),
        ChatMessage(role="user", content=question)
    ]
    answer = str(llm.chat(messages))
    return answer

def print_hyde_response(question):
    answer = create_hypothetical_answer(question)
    print("Hypothetical answer: ", answer)
    return print_response(answer)

In [15]:
print_hyde_response("What was Alexander's strategy against Darius III?");

Hypothetical answer:  assistant: Alexander's strategy against Darius III centered on decisive battles to cripple the Persian army and seize key territories. He aimed to force Darius into a direct confrontation, exploiting his own superior tactics and the Macedonian phalanx to achieve a crushing victory, thus undermining Darius's authority and control over his empire.



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Alexander received information from captured Persian scouts and rested his army for four days, fortifying the camp. He planned to leave behind baggage and unfit soldiers, advancing with warriors and weapons. Marching at night, he aimed to engage the enemy at daybreak. Darius drew out his army upon learning of Alexander's approach, but hills obscured their view. When only thirty stades apart, Alexander halted his phalanx and convened a council. He deliberated whether to attack immediately or encamp to reconnoiter the ground and enemy tactics, with Parmenio's advice to encamp prevailing. Alexander reconnoitered the area with light infantry and cavalry, then addressed his leaders, emphasizing their past valor and the significance of the upcoming battle for Asia's rule. He stressed discipline, silence when needed, and a sonorous battle-cry when opportune, urging them to swiftly obey and transmit orders, recognizing individual and collective responsibility for victory.

Node ID: d172a7db-ec

Answer for chunk size = 100:
<pre>
Hypothetical answer:  assistant: Alexander's strategy against Darius III centered on forcing decisive battles to cripple the Persian army and seize key territories. He aimed to draw Darius into open conflict, exploiting the Macedonian army's superior tactics and discipline to achieve victory and ultimately depose Darius from his throne.

Batches: 100%
 1/1 [00:00<00:00,  1.57it/s]
Alexander commanded in the field against Darius, pointing out the great rewards to be won. When Darius began to set his whole phalanx in motion, Alexander ordered an attack.

Node ID: c283ed80-e07e-4b79-9318-09a25b6976fa
Text: In addition to all this,  Alexander was commanding in the field
against Darius. These things  he enumerated as evidences of their
superiority in the struggle; and  then he began to point out the great
rewards they would win from the  danger to be incurred.
Score:  0.798

Node ID: 74b1f3d9-84d3-47e3-9597-793085c8e50f
Text: DEFEAT OF THE PERSIANS AND PURSUIT OF DARIUS.      At this
juncture, while the Macedonians were doubtful as to the result  of the
battle, Parmenio sent a messenger to Alexander in haste, to  tell him
that their side was in a critical position and that he  must send him
aid.
Score:  0.780

Node ID: e8f0aa7c-a261-4b33-bffd-798af571a8b1
Text: ix.). But Diodorus says that Alexander ordered  and arranged the
assault, that the Thebans made a brave and desperate  resistance for a
long time, and that not only the Boeotian allies, but  the Macedonians
themselves committed great slaughter of the besieged  (_Diod.
Score:  0.774

Node ID: 9ba1d49c-dc7d-4c09-90a0-5bcc4f4b6fa7
Text: BATTLE OF ARBELA.—FLIGHT OF DARIUS.      As soon as Darius began
to set his whole phalanx in motion, Alexander  ordered Aretes to
attack those who were riding completely round his  right wing; and up
to that time he was himself leading his men in  column.
Score:  0.768
</pre>

Answer for chunk size = 1024:
<pre>
Hypothetical answer:  assistant: Alexander's strategy against Darius III centered on forcing decisive battles to cripple the Persian army and seize key territories. He aimed to draw Darius into open combat, exploiting his own superior tactics and the Macedonian phalanx to achieve overwhelming victories, rather than relying on prolonged sieges or attrition.

Batches: 100%
 1/1 [00:00<00:00,  1.09it/s]
Alexander's army was arranged with the cavalry Companions on the right wing, followed by the royal squadron and other squadrons. The phalanx of Macedonian infantry was positioned near the cavalry, with select corps of shield-bearing guards leading the way. Darius's army was set up with Bactrian cavalry, Daans, and Arachotians on the left wing, Persians, Susians, and Cadusians near them, and men from Coele-Syria and Mesopotamia on the right. Medes, Parthians, Sacians, Tapurians, Hyrcanians, Albanians, and Sacesinians were also positioned on the right. In the center, where Darius was, were his kinsmen, Persian guards, Indians, Carians, and Mardian archers. The Uxians, Babylonians, men from the Red Sea, and Sitacenians were also in the center. Scythian cavalry, Bactrians, and scythe-bearing chariots were posted on the left, while Armenian and Cappadocian cavalry with scythe-bearing chariots were on the right. Greek mercenaries were stationed near Darius, opposite the Macedonian phalanx.

Node ID: 240ff12c-c7fd-4731-9750-b47b7b7c8707
Text: And it turned out  just as Alexander had conjectured; for as
soon as the battle became a  hand-to-hand one, the part of the Persian
army stationed on the left  wing was put to rout; and here Alexander
and his men won a brilliant  victory. But the Grecian mercenaries
serving under Darius attacked  the Macedonians at the point where they
saw their...
Score:  0.791

Node ID: 0be6d501-d59e-465f-8769-81180073c97d
Text: Moreover, if  any unexpected defeat befell his army, the
circumjacent country was  friendly to the enemy, and they were
acquainted with the locality,  whereas the Macedonians[401] were
unacquainted with it, and surrounded  by nothing but foes, of whom
there were a great number prisoners.  These would be a great source of
anxiety, as they would b...
Score:  0.781
</pre>

In [16]:
print_hyde_response("Describe the relationship between Alexander and Diogenes.");

Hypothetical answer:  assistant: Alexander the Great, known for his ambition and power, is said to have encountered the philosopher Diogenes, known for his simple and unconventional lifestyle. While accounts vary, the most famous anecdote describes Alexander asking Diogenes if he needed anything, to which Diogenes replied, "Stand out of my sun." This highlights the contrast between worldly power and philosophical independence.



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

I am sorry, but the provided text does not contain information about Alexander's encounter with Diogenes. However, the text does detail Alexander's interactions with Indian philosophers, including Dandamis and Calanus, and also describes a dispute between Callisthenes and Anaxarchus regarding the honors due to Alexander.

Node ID: 55632250-a239-415c-9f5c-e95ea278e072
Text: Again, when he arrived at Taxila and saw  the naked sect of
Indian philosophers, he was exceedingly desirous  that one of these
men should live with him; because he admired their  power of
endurance.[833] But the oldest of the philosophers, Dandamis  by name,
of whom the others were disciples, refused to come himself  to
Alexander, and would not...
Score:  0.765

Node ID: 5318d84d-c840-468f-9583-abfc4283e0a2
Text: There were  not wanting those who in regard to these matters
gave way to his wishes  with the design of flattering him; among
others being Anaxarchus, one  of the philosophers attending his court,
and Agis,

Answer for chunk size = 100:
<pre>
Hypothetical answer:  assistant: Alexander the Great, known for his ambition and power, is said to have encountered the philosopher Diogenes, known for his simple and unconventional lifestyle. While accounts vary, the most famous anecdote describes Alexander asking Diogenes if he needed anything, to which Diogenes replied, "Stand out of my sun." This highlights the contrast between worldly power and philosophical independence.

Batches: 100%
 1/1 [00:00<00:00,  1.21it/s]
Diogenes wanted nothing other than for Alexander and his attendants to move out of the sunlight. Alexander is known to have expressed his admiration for Diogenes’s conduct. Despite this, Alexander was still a slave to his insatiable ambition.

Node ID: 0f23358e-cc16-4532-8f17-535e66d5c650
Text: But Diogenes said that he  wanted nothing else, except that he
and his attendants would stand out  of the sunlight. Alexander is said
to have expressed his admiration  of Diogenes’s conduct.
Score:  0.844

Node ID: 2f31f92a-280d-4de6-85ef-a42c58549182
Text: Alexander is said to have expressed his admiration  of
Diogenes’s conduct.[832] Thus it is evident that Alexander was  not
entirely destitute of better feelings; but he was the slave of  his
insatiable ambition.
Score:  0.807

Node ID: fc3db266-194a-4c2c-bc61-be3ddb8ace0d
Text: Alexander refers to him and the Athenians. See Plutarch
(_Alex._, 55).    [566] Cf. _Arrian_ (vii. 29).    [567] _Curtius_
(viii.
Score:  0.765

Node ID: bf024ecd-0a39-48f3-8eac-adfb162a96b4
Text: [882] Literally “with his own head,” an Homeric expression. We
learn  from Plutarch (_Eumenes_, 6), that Craterus was a great
favourite with  the Macedonians because he opposed Alexander’s Asiatic
innovations.
Score:  0.762
</pre>

Answer for chunk size = 1024:
<pre>
Hypothetical answer:  assistant: Alexander the Great, known for his ambition and power, is said to have encountered the Cynic philosopher Diogenes, who lived in a large jar. Alexander, impressed by Diogenes's lack of material desires, reportedly asked if he could grant him any wish, to which Diogenes replied, "Stand out of my sun." This anecdote highlights the contrast between worldly power and philosophical detachment.

Batches: 100%
 1/1 [00:00<00:00,  1.03it/s]
Alexander met Diogenes of Sinope in the Isthmus and asked if he wanted anything. Diogenes replied that he wanted nothing other than for Alexander and his attendants to stand out of the sunlight. Alexander is said to have expressed his admiration of Diogenes’s conduct.

Node ID: 20e92215-fb3e-4978-bbb7-477daacd4b3b
Text: But this I think I can confidently  affirm, that he meditated
nothing small or mean; and that he would  never have remained
satisfied with any of the acquisitions he had made,  even if he had
added Europe to Asia, or the islands of the Britons to  Europe; but
would still have gone on seeking for unknown lands beyond  those
mentioned. I verily be...
Score:  0.793

Node ID: 9320067a-9eac-4a74-a111-90becb50e618
Text: There is also a current  report that Alexander wished men to
prostrate themselves before him as  to a god, entertaining the notion
that Ammon was his father, rather  than Philip; and that he now showed
his admiration of the customs of  the Persians and Medes by changing
the style of his dress, and by the  alteration he made in the general
etique...
</pre>

## Query transformation

Adding context to the query can help match the content of chunks better.

In [23]:
def add_context_to_query(question):
    messages = [
        ChatMessage(role="system", 
                    content="""The following question is about topics discussed in a 2nd century book about Alexander the Great.
                    Clarify the question posed in the following ways:
                    * Expand to include 2nd century names. For example, a question about Iranians should include answers about Parthians, Persians, Medes, Bactrians, etc.
                    * Provide context on terms. For example, that Ammonites came from Jordan or that Philip was the father of Alexander.
                    Provide only the clarified question without any preamble or instructions.
                    """
                   ),
        ChatMessage(role="user", content=question)
    ]
    expanded_question = str(llm.chat(messages))
    return expanded_question

def print_response_to_expanded_question(question):
    expanded_question = add_context_to_query(question)
    print("Expanded question: ", expanded_question)
    return print_response(expanded_question)

print_response_to_expanded_question("How did the Persian king react to Greeks' victory");

Expanded question:  assistant: How did the Arsacid (Parthian) or Achaemenid (Persian) king, the ruler of the Iranian peoples, react to the victories of the Greeks, specifically the Macedonians led by Alexander, son of Philip II?



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Darius, the king of Persia, sent a letter to Alexander requesting the return of his captured family and proposing a friendship and alliance. He acknowledged Alexander's military success but attributed it to the will of the gods.

In response, Alexander reminded Darius of the historical conflicts initiated by the Persians against the Macedonians and Greeks. He pointed out Darius's role in instigating the assassination of his father and his attempts to incite the Greeks to war. Alexander asserted his dominance over Asia due to his victories and offered to return Darius's family and grant any request he made. However, he demanded that Darius acknowledge him as the king of Asia and communicate with him accordingly.

Node ID: 64ea1229-b295-46c5-bcc3-70c8c2f1b017
Text: They were also instructed to support this petition  by word of
mouth. The letter pointed out to him that friendship and  alliance had
subsisted between Philip and Artaxerxes;[284] and that  when Arses,
son of Artaxerxes, ascen